In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
from input_configuration import base_year

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [5]:
output_dir = r'..\..\..\outputs\compare\RTP\person'
# if os.path.exists(output_dir):
#     files = glob.glob(output_dir)
#     for f in files:
#         os.remove(f)
# os.makedirs(output_dir)

# Trips per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [6]:
pd.options.display.float_format = '{:0,.1f}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\person_trips.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

### County and Region

In [7]:
def trips_per_day(geog): 
    df1 = df_person.groupby(geog).sum()[['psexpfac']]
    df2 = df_trip.groupby(geog).sum()[['trexpfac']]
    df = df1.merge(df2, left_index=True, right_index=True)
    df.loc['Region', 'psexpfac'] = df_person.sum()['psexpfac']
    df.loc['Region', 'trexpfac'] = df_trip.sum()['trexpfac']
    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']

    df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
    for purp in ['Work','Non-Work']:
        _df = df_trip[df_trip['trip_type'] == purp].groupby(geog).sum()[['trexpfac']]
        _df.rename(columns={'trexpfac': purp + ' Trips'}, inplace=True)

        # Add regional total to bottom of table
        _df.loc['Region', purp + ' Trips'] = df_trip[df_trip['trip_type'] == purp].sum()['trexpfac']
        df = df.merge(_df, left_index=True, right_index=True)

    df['Work Trips per Day'] = df['Work Trips']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work Trips']/df['psexpfac']
    df[['Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']]
    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [8]:
df = trips_per_day('hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_county.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.7,3.5,4.2
Kitsap,0.5,3.4,3.9
Pierce,0.5,3.3,3.8
Snohomish,0.6,3.4,3.9
Region,0.6,3.4,4.0


### Regional Growth Centers

In [9]:
df = trips_per_day('hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_rgc.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.5,3.5,4.0
Ballard-Interbay,1.0,3.6,4.6
Bellevue,0.9,3.6,4.6
Bothell Canyon Park,0.8,3.3,4.2
Bremerton,0.6,3.5,4.1
Burien,0.7,3.4,4.2
Cascade,0.5,3.2,3.7
Duwamish,0.8,3.5,4.3
Everett,0.6,3.5,4.2


### Regional Geographies

In [10]:
df = trips_per_day('hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_reg_geog.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
CitiesTowns,0.5,3.4,3.9
Core,0.6,3.4,4.0
HCT,0.6,3.4,4.0
Metro,0.7,3.6,4.3
Rural,0.5,3.2,3.7
UU,0.5,3.3,3.8
Region,0.6,3.4,4.0


### Equity Geographies

In [11]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 1].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Work','Non-Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
results_df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
df

Trip Type,Non-Work,Work
Equity Group,,
hh_elderly_50,3.5,0.3
hh_elderly_reg,3.4,0.6
hh_english_reg,3.5,0.6
hh_poverty_50,3.4,0.5
hh_poverty_reg,3.4,0.6
hh_racial_50,3.5,0.6
hh_racial_reg,3.5,0.6
hh_youth_reg,3.4,0.5


# Miles Driven per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [12]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

### County

In [13]:
df = vmt_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_county.csv')); incr+=1
df

,Average Miles per Person
hh_county,
King,15.9
Kitsap,15.2
Pierce,17.7
Snohomish,19.8
Region,17.0


### Regional Growth Centers

In [14]:
df = vmt_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_rgc.csv')); incr+=1
df

,Average Miles per Person
hh_rgc,
Auburn,12.9
Ballard-Interbay,10.4
Bellevue,7.1
Bothell Canyon Park,20.1
Bremerton,5.4
Burien,14.0
Cascade,17.0
Duwamish,10.6
Everett,9.0


### Regional Geography

In [15]:
df = vmt_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_reg_geog.csv')); incr+=1
df

,Average Miles per Person
hh_rg_proposed,
CitiesTowns,21.6
Core,16.3
HCT,17.3
Metro,11.6
Rural,26.0
UU,19.3
Region,17.0


### Equity Geographies

In [16]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = vmt_per_person(df_vmt, df_person, col)
        if 1 in df.index:
            df = df.loc[1]
        df['source'] = col
        result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_equity.csv')); incr+=1
result_df

,Average Miles per Person
source,
hh_youth_reg,18.7
hh_youth_50,17.0
hh_youth_50,17.0
hh_elderly_reg,17.5
hh_elderly_50,27.0
hh_english_reg,15.6
hh_english_50,17.0
hh_english_50,17.0
hh_racial_reg,15.1


# Miles Walking and Biking per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [17]:
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')



def walk_bike_per_person(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
        _df_person = df_person.groupby(geog).sum()[['psexpfac']]

        df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

### County

In [18]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,0.5,0.3,0.8
Kitsap,0.4,0.2,0.7
Pierce,0.4,0.3,0.7
Snohomish,0.4,0.3,0.7
Region,0.5,0.3,0.8


### Regional Growth Center 

In [19]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_rgc.csv'))
df

,Walk,Bike,Walk and Bike
hh_rgc,,,
Auburn,0.6,0.5,1.1
Ballard-Interbay,0.8,0.4,1.2
Bellevue,1.0,0.3,1.2
Bothell Canyon Park,0.5,0.6,1.1
Bremerton,0.8,0.3,1.1
Burien,0.5,0.4,0.9
Cascade,0.5,0.2,0.7
Duwamish,0.8,0.3,1.1
Everett,0.8,0.3,1.0


### Regional Geography

In [20]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_reg_geog.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,0.4,0.3,0.6
Core,0.4,0.3,0.7
HCT,0.4,0.3,0.7
Metro,0.7,0.3,1.0
Rural,0.3,0.3,0.6
UU,0.4,0.2,0.6
Region,0.5,0.3,0.8


### Equity Geographies

In [21]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = vmt_per_person(df_vmt, df_person, col)
        if 1 in df.index:
            df = df.loc[1]
        df['source'] = col
        result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Average Miles per Person
source,
hh_youth_reg,25.2
hh_youth_50,23.1
hh_youth_50,23.1
hh_elderly_reg,23.6
hh_elderly_50,32.8
hh_english_reg,21.9
hh_english_50,23.1
hh_english_50,23.1
hh_racial_reg,21.2


# Jobs Accessible within 45 Minutes of Transit

In [22]:
pd.options.display.float_format = '{:0,.0f}'.format
df = pd.read_csv(r'..\..\..\outputs\access\transit_jobs_access.csv'); incr+=1

### Region

In [23]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_region.csv')); incr+=1
_df

,Jobs within 45-minute Transit Commute
geography,
1,"135,741"


### County

In [24]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_county.csv')); incr+=1
_df

,Jobs within 45-minute Transit Commute
geography,
King,"228,896"
Kitsap,"3,846"
Pierce,"21,241"
Snohomish,"27,591"


### Regional Growth Center

In [25]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_rgc.csv')); incr+=1
_df

,Jobs within 45-minute Transit Commute
geography,
Auburn,"59,002"
Ballard-Interbay,"515,353"
Bellevue,"468,374"
Bothell Canyon Park,"42,381"
Bremerton,"28,963"
Burien,"186,334"
Cascade,"6,107"
Duwamish,"497,662"
Everett,"64,308"


### Regional Geography

In [26]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_reg_geog.csv')); incr+=1
_df

,Jobs within 45-minute Transit Commute
geography,
CitiesTowns,"14,901"
Core,"57,873"
HCT,"38,264"
Metro,"335,642"
Rural,386
UU,"8,126"


### Equity Geographies

In [27]:
geo_list = []
equity_geogs = ['youth','elderly','english','racial','poverty']
for equity_geog in equity_geogs:
    for geog_type in ['_geog_vs_reg_total','_geog_vs_50_percent']:
        geo_list.append(equity_geog+geog_type)

In [28]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df

,geography_group,value
50,youth_geog_vs_reg_total,"49,741"
55,elderly_geog_vs_reg_total,"108,023"
58,elderly_geog_vs_50_percent,0
61,english_geog_vs_reg_total,"124,923"
66,racial_geog_vs_reg_total,"149,052"
69,racial_geog_vs_50_percent,"148,367"
72,poverty_geog_vs_reg_total,"134,151"
75,poverty_geog_vs_50_percent,"221,960"


# Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.
Average accessible jobs are weighted averages based on parcel household population.

In [29]:
df = pd.read_csv(r'..\..\..\outputs\access\walk_bike_jobs_access.csv')

### Regional Average

In [30]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
_df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike
geography_value,,
1,"9,609","50,985"


### Home County

In [31]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_county.csv')); incr+=1
_df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike
geography_value,,
King,"15,837","80,060"
Kitsap,"1,145","6,990"
Pierce,"2,190","16,116"
Snohomish,"1,999","17,250"


### Regional Growth Center

In [32]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_rgc.csv')); incr+=1
_df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike
geography_value,,
Auburn,"11,555","41,672"
Ballard-Interbay,"13,278","80,607"
Bellevue,"52,917","105,281"
Bothell Canyon Park,"7,779","19,229"
Bremerton,"12,005","29,786"
Burien,"4,355","13,306"
Cascade,"1,670","8,580"
Duwamish,"28,832","248,710"
Everett,"15,432","35,116"


### Regional Geography

In [33]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_reg_geog.csv')); incr+=1
_df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike
geography_value,,
CitiesTowns,875,"7,225"
Core,"2,956","27,722"
HCT,"1,349","14,455"
Metro,"25,391","120,825"
Rural,105,"1,716"
UU,350,"5,825"


### Equity Geographies

In [34]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df

,geography_group,jobs_1_mile_walk,jobs_3_mile_bike
50,youth_geog_vs_reg_total,"1,669","19,740"
55,elderly_geog_vs_reg_total,"6,765","35,511"
58,elderly_geog_vs_50_percent,91,"1,528"
61,english_geog_vs_reg_total,"6,324","42,317"
66,racial_geog_vs_reg_total,"13,776","61,381"
69,racial_geog_vs_50_percent,"7,059","51,099"
72,poverty_geog_vs_reg_total,"11,804","54,893"
75,poverty_geog_vs_50_percent,"16,335","58,038"
